In [1]:
import pandas as pd
import numpy as np

In [5]:
predict_df = pd.read_csv('../submit/20221101_030308_FFDCN.csv')
predict_df

,user_id,isbn,rating
0,11676,0002005018,6.964805
1,116866,0002005018,9.159520
2,152827,0060973129,7.524877
3,157969,0374157065,6.336005
4,67958,0399135782,8.511495
...,...,...,...
76694,278543,1576734218,6.565018
76695,278563,3492223710,7.418255
76696,278633,1896095186,5.116924
76697,278668,8408044079,2.064817


In [6]:
predict_df.loc[predict_df['rating']>10,'rating']=10
predict_df.loc[predict_df['rating']<1,'rating']=1

In [7]:
predict_df.to_csv('../submit/20221101_030308_FFDCN.csv',index=False)

In [2]:
books = pd.read_csv('../data/books.csv')
users = pd.read_csv('../data/users.csv')
ratings = pd.read_csv('../data/train_ratings.csv')
test_ratings = pd.read_csv('../data/test_ratings.csv')

In [11]:
users.nunique()

user_id     68092
location    18368
age            91
dtype: int64

In [5]:
ratings

,user_id,isbn,rating
0,8,0002005018,4
1,67544,0002005018,7
2,123629,0002005018,8
3,200273,0002005018,8
4,210926,0002005018,9
...,...,...,...
306790,278843,0743525493,7
306791,278851,067161746X,6
306792,278851,0884159221,7
306793,278851,0912333022,7


In [9]:
test_users = test_ratings['user_id'].unique()

In [3]:
df_count = pd.DataFrame(ratings['isbn'].value_counts().reset_index())
df_count.columns=['isbn','count']
df_count

,isbn,count
0,0316666343,566
1,0971880107,465
2,0385504209,390
3,0312195516,307
4,0060928336,256
...,...,...
129772,1854710443,1
129773,1853915262,1
129774,1853361208,1
129775,1853263893,1


In [6]:
df_book_rating = pd.merge(books,df_count,how='left',on='isbn')[['isbn','count']]
df_book_rating

,isbn,count
0,0002005018,7.0
1,0060973129,1.0
2,0374157065,5.0
3,0399135782,14.0
4,0425176428,1.0
...,...,...
149565,067161746X,1.0
149566,0767907566,NaN
149567,0884159221,1.0
149568,0912333022,1.0


In [7]:
df_book_rating.loc[df_book_rating['count'].isna(),'count'] = 0
df_book_rating[df_book_rating['count']<10]

,isbn,count
0,0002005018,7.0
1,0060973129,1.0
2,0374157065,5.0
4,0425176428,1.0
5,0671870432,0.0
...,...,...
149565,067161746X,1.0
149566,0767907566,0.0
149567,0884159221,1.0
149568,0912333022,1.0


In [8]:
df_book_rating = df_book_rating.astype({'count':int})

In [9]:
df_book_rating[df_book_rating['count']==0]

,isbn,count
5,0671870432,0
21,0689821166,0
52,0245542957,0
81,0375509038,0
134,1900850303,0
...,...,...
149546,1576734218,0
149551,3492223710,0
149556,1896095186,0
149561,8408044079,0


In [10]:
df_book_rating

,isbn,count
0,0002005018,7
1,0060973129,1
2,0374157065,5
3,0399135782,14
4,0425176428,1
...,...,...
149565,067161746X,1
149566,0767907566,0
149567,0884159221,1
149568,0912333022,1


In [11]:
df_book_rating.to_csv('../data/book_review_count.csv')

In [40]:
def cold_start_set(n: int=0) -> set():
    df = pd.read_csv('../data/user_review_count.csv')
    cold_user = set(df[df['count']<=n]['user_id'])
    return cold_user

In [76]:
df[df['user_id']==26540]

,user_id,isbn,rating,location,age,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path,uid,iid
26390,26540,0060928336,7,"jacksonville, florida, usa",NaN,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Rebecca Wells,1997.0,Perennial,http://images.amazon.com/images/P/0060928336.0...,en,['Fiction'],A NOVEL ABOUT THE COMPLEX BONDS BETWEEN A MOTH...,images/0060928336.01.THUMBZZZ.jpg,13701,1310
38641,26540,0671024248,6,"jacksonville, florida, usa",NaN,Hearts In Atlantis,Stephen King,2000.0,Pocket,http://images.amazon.com/images/P/0671024248.0...,en,['Fiction'],"In the title story, a bunch of college kids ge...",images/0671024248.01.THUMBZZZ.jpg,13701,2192
87186,26540,0345449347,1,"jacksonville, florida, usa",NaN,The Fourth Hand,John Irving,2002.0,Ballantine Books,http://images.amazon.com/images/P/0345449347.0...,en,['Fiction'],When a New York journalist suffers a horrible ...,images/0345449347.01.THUMBZZZ.jpg,13701,8699


In [136]:
user_review_count = pd.read_csv('../data/user_review_count.csv',index_col=0)
user_review_count.sort_values(by='count').iloc[63195:]

,user_id,count
80,7352,10
23917,68621,10
7111,106406,10
11401,169600,11
20649,165669,11
...,...,...
821,23902,956
553,153662,1496
4602,189835,1503
3856,98391,4560


In [137]:
cold_users = cold_start_set(10)

In [138]:
len(cold_users)

63198

In [95]:
for i,j,k in df_user_rating.itertuples():
    print(j)
    break

8


In [ ]:

print(X_valid[:5])
print(y_valid[:5])
pd.DataFrame({'user_id':X_valid['user_id'].map(data['idx2user']).values,'isbn':X_valid['isbn'].map(data['idx2isbn']).values,'rating':y_valid.values}).to_csv('./data/valid_ratings.csv',index=False)
raise Exception('Done')

## MF를 위한 Data 전처리 및 모델링 테스트

In [37]:
import numpy as np
import pandas as pd
import scipy
import random
import math
from scipy.sparse import csr_matrix, linalg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [87]:
books = pd.read_csv('../data/books.csv')
users = pd.read_csv('../data/users.csv')
ratings = pd.read_csv('../data/train_ratings.csv')
test_ratings = pd.read_csv('../data/test_ratings.csv')

In [1]:
trainbook = set(ratings['isbn'].unique())
testbook = set(test_ratings['isbn'].unique())

NameError: name 'ratings' is not defined

In [99]:
print(f"train 책 데이터는 {len(trainbook)}개 입니다.")
print(f"test 책 데이터는 {len(testbook)}개 입니다.")
print(f"train과 test 둘 다에 있는 책 데이터는 {len(testbook&trainbook)}개 입니다.")
print(f"test에는 있지만 train에는 없는 책 데이터는{len(testbook-trainbook)}개 입니다.")

train 책 데이터는 129777개 입니다.
test 책 데이터는 52000개 입니다.
train과 test 둘 다에 있는 책 데이터는 32207개 입니다.
test에는 있지만 train에는 없는 책 데이터는19793개 입니다.


In [100]:
trainuser = set(ratings['user_id'].unique())
testuser = set(test_ratings['user_id'].unique())

In [103]:
print(f"train 유저 데이터는 {len(trainuser)}명 입니다.")
print(f"test 유저 데이터는 {len(testuser)}명 입니다.")
print(f"train과 test 둘 다에 있는 유저 데이터는 {len(testuser&trainuser)}명 입니다.")
print(f"test에는 있지만 train에는 없는 유저 데이터는{len(testuser-trainuser)}명 입니다.")
print(f"train과 test 둘 다에 있는 유저 데이터는 {len(testuser.union(trainuser))}명 입니다.")

train 유저 데이터는 59803명 입니다.
test 유저 데이터는 26167명 입니다.
train과 test 둘 다에 있는 유저 데이터는 17901명 입니다.
test에는 있지만 train에는 없는 유저 데이터는8266명 입니다.
train과 test 둘 다에 있는 유저 데이터는 68069명 입니다.


In [ ]:
drop_ratings = ratings.copy()
drop_ratings

In [34]:
seed=42

In [35]:
books['isbn'] = books['img_url'].apply(lambda x: x.split('P/')[1][:10])

In [36]:
user2idx = {v:k for k,v in enumerate(ratings['user_id'].unique())}
book2idx = {v:k for k,v in enumerate(ratings['isbn'].unique())}

ratings['isbn'] = ratings['isbn'].map(book2idx)
ratings['user_id'] = ratings['user_id'].map(user2idx)
ratings

,user_id,isbn,rating
0,0,0,4
1,1,0,7
2,2,0,8
3,3,0,8
4,4,0,9
...,...,...,...
306790,6313,129772,7
306791,1879,129773,6
306792,1879,129774,7
306793,1879,129775,7


In [38]:
X_train, X_test_tmp, y_train, y_test_tmp = train_test_split(ratings[['user_id', 'isbn']], ratings['rating'], test_size=0.4, random_state=seed)
X_valid, X_test, y_valid, y_test = train_test_split(X_test_tmp, y_test_tmp, test_size=0.5, random_state=seed)

In [39]:
train = np.zeros((len(user2idx), len(book2idx)))
valid = np.zeros((len(user2idx), len(book2idx)))
test = np.zeros((len(user2idx), len(book2idx)))

MemoryError: Unable to allocate 57.8 GiB for an array with shape (59803, 129777) and data type float64